# Step 5: Feature Engineering

Feature engineering is performed selectively based on insights from exploratory data analysis and statistical validation. 
The goal is to strengthen behavioral fraud signals while maintaining interpretability and avoiding data leakage.

Only domain-relevant and EDA-supported features are engineered. No automated or high-dimensional transformations are applied.


In [1]:
# load 
import pandas as pd
import numpy as np

# Load preprocessed dataset
df = pd.read_csv("../data/processed/cleaned_data.csv")

df.shape


(15420, 74)